## 5. RNN을 활용한 일기 생성 

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from text_loader import TextLoader 

In [2]:
# 데이터 불러오기

num_layers  = 3
hidden_size = 512
batch_size  = 1
max_length  = 1000

loader = TextLoader("data/football_diary_normalization.txt")
vocab_size = len(loader.vocab)

In [3]:
X = tf.placeholder(tf.int32, [None, None])
x_one_hot = tf.one_hot(X, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, states = tf.nn.dynamic_rnn(cells, x_one_hot, 
                                    initial_state=initial_state, dtype=tf.float32)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.fully_connected(outputs, vocab_size,
                                activation_fn=None)
y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [6]:
# 문장 하나씩을 입력값으로 넣고, 일기 생성 파악

sentence = list("역전승을 거두었다")

print("Start with:", "".join(sentence))
sentence = [loader.vocab[word] for word in sentence]

saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "char-rnn_116000")
    
    # 매 이터레이션마다 글자 하나씩 생성
    pred_char, state = sess.run([pred, states], feed_dict={X:[sentence[:]]})
    for i in range(1000):
        pred_char, state = sess.run([pred, states], 
            feed_dict={X:[[sentence[-1]]], initial_state: state}) # 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌
        sentence.append(pred_char[0][-1])
        
sentence = [loader.words[char] for char in sentence]
print("".join(sentence))

Start with: 역전승을 거두었다
INFO:tensorflow:Restoring parameters from char-rnn_116000
역전승을 거두었다 키미에서 남았다. 호주는 패스를 잘하는데……. 엄마, 아빠께서는 이란 편을 해 주었다. 내 동생과 나는 호주를 응원하는데……. 그런데 이란이 한골을 허용하였다. 이란이 공격을 더 해왔다. 그래서 2골이 허용되었다. 시간은 좀 남았다. 2-2로 비길 경우 이란이 나간다고 하는데……. 시간이 많이 지나갔다. 주심이 호루라기를 불었다. 이란이 본선 티켓을 탄 것이다. 마지막 티켓을 쥔 이란. 참 좋겠다.
일요일에 서정원 선수 아저씨의 꿈을 꾸었다. 지난 번에는 또 최용수 선수 아저씨의 꿈을 꾸었었다. 하지만 지금은 서정원 선수 아저씨의 꿈을 말하겠다. 서정원 선수 아저씨가 우리 피아노 학원에 다니는데 아이들이 서정원 선수 아저씨가 수고했다고 편지를 주었다. 나도 편지를 주었다. 하루는 내가 피아노 학원에 갔다. 그런데 서정원 선수 아저씨가 원장 선생님께 레슨을 받고 있었다. 서정원 선수 아저씨가 “현아야!” 라고 불렀다. 그런데 일어나보니 엄마께서 부르는 소리였다. ‘이게 뭔 꿈인다냐?’ 꿈 꿀 동안에 참 재미있었다.
1월 25일 일요일 바로 오늘 방콕에서 한국 국가 선수 아저씨들이 덴마크와 축구를 한다. 그런데 하는 것을 보여주지 않는다. 그 이유는 경제가 어렵기 때문이다. 스포츠 뉴스 아나운서가 지금은 한국이 덴마크에게 1점을 주었다고 말하였다. 중계는 안하지만 한국이 2대1이라도 이겼으면 좋겠다. 중계를 했으면 엄청 좋겠었는데……. IMF시대를 빨리 극복을 하여서 축구중계를 볼 수 있게 했으면 좋겠다. 또 이유가 있다. 영국의 방송국에서 독점중계를 하면서 축구대회의 중계방송하는 비용을 한국의 방송국에 너무 많이 내라고 돈을(달러$) 요구했기 때문에 우리나라 방송국 취소를 했다. 아마도 IMF 때문에 그런 것 같다. 다음에 IMF를 극복하서 꼭 축구중계를 했으면 하늘에 날아갈 것 같은 기분일 것이다.
오

### => RNN을 통한 일기 생성 분석

> * 상대적으로 적은 양의 데이터로 인해 짧은 시간에도 불구하고 상대적으로 빅데이터들보다는 많은 학습을 할 수 있었다. 
> * 내용에 있어서는 문맥은 맞고 내용도 맞으나 전체적인 맥락들은 아직 맞지 않는다는 것이 한계점이라고 말할 수 있다. 